# Obtaining and Cleaning the Data
### This notebook contains the steps taken to obtain the raw data either from an excel spreadsheet taken from the FEC website or the OpenFEC API. Data Cleaning steps were also performed in this notebook.

In [1]:
import requests
import json
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from ratelimit import limits, sleep_and_retry

%matplotlib inline

In [ ]:
myapikey = 'INSERT API KEY HERE'

In [2]:
year = 2016
one_hour = 3600 # seconds/hour

In [3]:
def states_and_districts():
# number of Congressional Districts per state after the 2010 census 
# 2013 was the first year with new districts

    return {
        'AL':7,
        'AK':1,
        'AZ':9,
        'AR':4,
        'CA':53,
        'CO':7,
        'CT':5,
        'DE':1,
        'FL':27,
        'GA':14,
        'HI':2,
        'ID':2,
        'IL':18,
        'IN':9,
        'IA':4,
        'KS':4,
        'KY':6,
        'LA':6,
        'ME':2,
        'MD':8,
        'MA':9,
        'MI':14,
        'MN':8,
        'MS':4,
        'MO':8,
        'MT':1,
        'NE':3,
        'NV':4,
        'NH':2,
        'NJ':12,
        'NM':3,
        'NY':27,
        'NC':13,
        'ND':1,
        'OH':16,
        'OK':5,
        'OR':5,
        'PA':18,
        'RI':2,
        'SC':7,
        'SD':1,
        'TN':9,
        'TX':36,
        'UT':4,
        'VT':1,
        'VA':11,
        'WA':10,
        'WV':3,
        'WI':8,
        'WY':1
    }

## Obtaining the 2016 House Election Results

In [5]:
# House of Reps 2016 Election Results 
# source: https://transition.fec.gov/pubrec/electionresults.shtml
house_results_2016 = pd.read_excel('federalelections2016.xlsx', sheet_name=12)

In [6]:
# clean up column names
house_results_2016.columns = house_results_2016.columns.str.strip()\
        .str.replace(' ','_').str.upper()

# drop first (0-th) column 
house_results_2016.drop(labels=np.nan, axis=1, inplace=True)

# rename columns for increased clarity
house_results_2016.rename({'FEC_ID#':'CANDIDATE_ID',
                           '(I)':'INCUMBENT'}, axis=1, inplace=True)

# remove trailing whitespace in CANDIDATE_ID column - needed in order to properly merge on later
house_results_2016.loc[:, 'CANDIDATE_ID'] =\
    house_results_2016.loc[:, 'CANDIDATE_ID'].str.strip()

# 1 == incumbent, 0 == otherwise
house_results_2016['INCUMBENT'].fillna('0',inplace=True)
house_results_2016.loc[:, 'INCUMBENT'] =\
    house_results_2016.loc[:, 'INCUMBENT'].str.replace('(','').str.replace(')','')\
        .str.replace('I','1').astype(int)

# 1 == winner, 0 == otherwise
house_results_2016['GE_WINNER_INDICATOR'].fillna('0',inplace=True)
house_results_2016.loc[:, 'GE_WINNER_INDICATOR'] =\
    house_results_2016.loc[:, 'GE_WINNER_INDICATOR'].str.replace('W','1').astype(int)

In [7]:
# drop all Special Election results which coincided with the General Election 
# HI-01, KY-01, PA-02
mask = house_results_2016['D'].str.contains(r'unexpired', flags=re.I, na=False)
house_results_2016.drop(house_results_2016[mask].index, inplace=True)

# drop the 'FULL TERM' label from 'D' (district) labels
# HI-01, KY-01, PA-02
mask = house_results_2016['D'].str.contains(r'full term', flags=re.I, na=False)
house_results_2016.loc[house_results_2016[mask].index, 'D'] =\
    house_results_2016[mask]['D'].str.replace(' - FULL TERM', '')

In [8]:
# Clean up names for the top 10 political parties present in this election cycle
# for more information see sheet 13 (0-indexed) of federalelections2016.xlsx

# replace variations of 'R' (with extra whitespace) with 'R' == Republican
mask = house_results_2016['PARTY'].str.contains(r'R[\s]+', flags=re.I, na=False)
house_results_2016.loc[house_results_2016[mask].index, 'PARTY'] =\
    house_results_2016[mask]['PARTY'].str.replace(r'R[\s]+', 'R')

# replaces variations of 'D' (with extra whitespace) with 'D' == Democratic
mask = house_results_2016['PARTY'].str.contains(r'D[\s]+', flags=re.I, na=False)
house_results_2016.loc[house_results_2016[mask].index, 'PARTY'] = \
    house_results_2016[mask]['PARTY'].str.replace(r'D[\s]+', 'D')

# replaces variations of 'W' (with extra whitespace) with 'W' == Write-In
mask = house_results_2016['PARTY'].str.contains(r'W[\s]+', flags=re.I, na=False)
house_results_2016.loc[house_results_2016[mask].index, 'PARTY'] = \
    house_results_2016[mask]['PARTY'].str.replace(r'W[\s]+', 'W')

# replaces variations of 'LIB' (with extra whitespace) with 'LIB' == Libertarian
mask = house_results_2016['PARTY'].str.contains(r'LIB[\s]+', flags=re.I, na=False)
house_results_2016.loc[house_results_2016[mask].index, 'PARTY'] = \
    house_results_2016[mask]['PARTY'].str.replace(r'LIB[\s]+', 'LIB')

party_mapper = {
    'W(R)' : 'R',
    'R/W' : 'R',
    'R/TRP' : 'R',
    'W(R)/R' : 'R',
    'R/IP' : 'R',
    'IP/R' : 'R',
    'R/CON' : 'R',
    'W(D)' : 'D',
    'W(D)/D' : 'D',
    'D/IP' : 'D',
    'D/R' : 'D',    # Peter Welch (D)
    'D/PRO/WF/IP' : 'D',    # Peter A. DeFazio (D)
    'W(D)/W' : 'D',
    'W(LIB)' : 'LIB',
    'W(GRE)' : 'GRE',
    'W(GRE)/GRE' : 'GRE',
    'W(IND)' : 'IND',
    'W(NOP)' : 'NOP'
}

# replace party labels for candidates with multiple parties
# with standard party designation 'R', 'D', 'LIB', etc.
house_results_2016.replace(party_mapper, inplace=True)

In [9]:
# replacing all non-digits with the primary votes and general votes columns with NaN 
# and converting the columns to floats
house_results_2016['PRIMARY_VOTES'] = house_results_2016['PRIMARY_VOTES'].astype(str)
house_results_2016['GENERAL_VOTES'] = house_results_2016['GENERAL_VOTES'].astype(str)

house_results_2016.loc[:, 'PRIMARY_VOTES'] =\
        house_results_2016.loc[:, 'PRIMARY_VOTES']\
            .str.replace('Unopposed', str(np.nan))\
            .str.replace('*', str(np.nan)).astype(float)

house_results_2016.loc[:, 'GENERAL_VOTES'] =\
        house_results_2016.loc[:, 'GENERAL_VOTES']\
            .str.replace('Unopposed', str(np.nan)).astype(float)

In [10]:
# extract the raw totals for each election
# (for potential sanity check later)
total_votes = house_results_2016\
                [house_results_2016['TOTAL_VOTES'].notnull()].copy()

In [11]:
# For candidates with multiple entries in the same primary election, select the row 
# with the higest number of primary votes.
# Most candidates who appear more than once are they are running in states which
# allow for Electoral Fusion.
# For more information, see: https://en.wikipedia.org/wiki/Electoral_fusion#United_States

nunique_cands_pre = pd.DataFrame()
nunique_cands_post = pd.DataFrame()

for state in states_and_districts():
    n_cand = house_results_2016[house_results_2016['STATE_ABBREVIATION'] == state]\
                ['CANDIDATE_ID'].nunique()
    
    nunique_cands_pre = nunique_cands_pre.append({'State' : state,
                                                  'Num. of Unique Candidates' : n_cand}, 
                                                ignore_index=True)

# drop duplicate candidate IDs
house_results_2016 = house_results_2016.sort_values('PRIMARY_VOTES', ascending=False)\
                        .drop_duplicates('CANDIDATE_ID')

for state in states_and_districts():
    n_cand = house_results_2016[house_results_2016['STATE_ABBREVIATION'] == state]\
                ['CANDIDATE_ID'].nunique()
    
    nunique_cands_post = nunique_cands_post.append({'State' : state,
                                                    'Num. of Unique Candidates' : n_cand}, 
                                                    ignore_index=True)

# check that the number of unique candidates per state is the same before and after
# dropping duplicates
print(nunique_cands_post.equals(nunique_cands_pre))

True


In [12]:
# For the states with Electoral Fusion, use the combined general election totals 
# (for all parties a candidate was voted for under) instead of general election 
# totals per party.

house_results_2016['GENERAL_VOTES'].mask(
            house_results_2016['COMBINED_GE_PARTY_TOTALS_(CT,_NY,_SC)'].notnull(),
            house_results_2016['COMBINED_GE_PARTY_TOTALS_(CT,_NY,_SC)'],
            inplace=True)

house_results_2016['GENERAL_%'].mask(
            house_results_2016['COMBINED_%_(CT,_NY,_SC)'].notnull(),
            house_results_2016['COMBINED_%_(CT,_NY,_SC)'],
            inplace=True)

In [37]:
house_results_2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 519 to 4058
Data columns (total 22 columns):
STATE_ABBREVIATION                       1971 non-null object
STATE                                    1971 non-null object
D                                        1970 non-null object
CANDIDATE_ID                             1970 non-null object
INCUMBENT                                1971 non-null int64
CANDIDATE_NAME_(FIRST)                   1967 non-null object
CANDIDATE_NAME_(LAST)                    1969 non-null object
CANDIDATE_NAME                           1970 non-null object
TOTAL_VOTES                              1 non-null object
PARTY                                    1969 non-null object
PRIMARY_VOTES                            1468 non-null float64
PRIMARY_%                                1466 non-null float64
RUNOFF_VOTES                             12 non-null float64
RUNOFF_%                                 12 non-null float64
GENERAL_VOTES              

## Columns to Keep and those to Remove
There are a few columns which won't be considered or needed for the analysis to follow, namely `COMBINED_GE_PARTY_TOTALS_(CT,_NY,_SC)` and `COMBINED_%_(CT,_NY,_SC)` since those values were copied over to `GENERAL_VOTES` and `GENERAL_%` respectively. The `RUNOFF_VOTES`, `RUNOFF_%`, `GE_RUNOFF_ELECTION_VOTES_(LA)` and `GE_RUNOFF_ELECTION_%_(LA)` will be dropped in order to simplify the analysis (`GE_WINNER_INDICATION` is the important metric).
Lastly, `STATE`, `CANDIDATE_NAME_(FIRST)`, ` CANDIDATE_NAME_(LAST)`, `TOTAL_VOTES`, 
and `FOOTNOTES` are either captured by other columns or unneeded altogether.

In [35]:
# columns to keep
results_cols = ['STATE_ABBREVIATION', 'D', 'INCUMBENT', 'CANDIDATE_NAME', 'CANDIDATE_ID', 
                'PARTY', 'PRIMARY_VOTES', 'PRIMARY_%', 'GENERAL_VOTES', 'GENERAL_%',
                'GE_WINNER_INDICATOR']

In [36]:
# final 2016 House Election Results dataframe
house_results_2016_clean =\
    house_results_2016[house_results_2016['CANDIDATE_ID'].notnull()][results_cols].copy()

In [15]:
house_results_2016_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1970 entries, 3200 to 4058
Data columns (total 9 columns):
STATE_ABBREVIATION     1970 non-null object
D                      1969 non-null object
INCUMBENT              1970 non-null int64
CANDIDATE_NAME         1970 non-null object
CANDIDATE_ID           1970 non-null object
PARTY                  1969 non-null object
PRIMARY_VOTES          1467 non-null float64
GENERAL_VOTES          1176 non-null float64
GE_WINNER_INDICATOR    1970 non-null int64
dtypes: float64(2), int64(2), object(5)
memory usage: 153.9+ KB


In [16]:
# save the dataframes as a CSV file
house_results_2016_clean.to_csv('house_results_2016_clean.csv', index=False)
total_votes.to_csv('total_votes_house_2016.csv', index=False)

## Obtaining Principal Committee IDs 
#### Search for principal committee IDs for each candidate ID that appears in the House Election Results dataframe 

In [17]:
# Search for Principal Committee ID for each candidate ID. 

# used to throttle API calls to FEC (limit 1000 per hour)
@sleep_and_retry
@limits(calls=15, period=60)
def url_call_search(candidate_id, api_key=myapikey):
    
    url = (
            'https://api.open.fec.gov/v1/'
            'candidates/search/?'
            'sort=name'
            '&api_key={api_key}'
            '&candidate_id={c}'
            '&page=1'
            '&per_page=50'
            '&sort_nulls_last=false'
            '&sort_hide_null=false'
            '&election_year=2016'
            '&office=H'
            '&sort_null_only=false'.format(api_key=myapikey,
                                            c=candidate_id)
            )

    # 'requests.models.Response' type returned
    r = requests.get(url)

    # verify URL
    if r.status_code != 200:
        raise Exception('API response: {}'.format(r.status_code)) 

    # convert 'Response' object to dict
    r_dict = r.json()

    # convert dict to a Dataframe object and return Dataframe
    return pd.DataFrame.from_dict(r_dict['results']) 

In [18]:
# Since the information returned from the API search call contains a lot more information than
# needed, the Principal Committee ID needs to be carefully extracted.
def get_principal_committees(df_results):
    # 'df_results' is a dataframe with the election results, contains Candidate IDs
    # return a dataframe with principal committee IDs for each candidate 
    
    search = pd.DataFrame()
    
    # query FEC API for each candidate
    for cand in df_results['CANDIDATE_ID']:
        search = search.append(url_call_search(cand), sort=True, ignore_index=True)

    size = search.shape[0]

    principal_committees = pd.DataFrame()

    principal_committees['CANDIDATE_ID'] = np.nan
    principal_committees['COMMITTEE_ID'] = np.nan

    # campaign committees for candidates for the US House of Representatives 
    # see codes and descriptions: 
    # https://www.fec.gov/campaign-finance-data/committee-type-code-descriptions/
    principal_committees['COMMITTEE_TYPE'] = np.nan

    # extract the principal committee id
    for idx in search.index.tolist():

        # number of principal committees for each candidate
        n_pc = len(search['principal_committees'][idx])

        for n in range(n_pc):
            cycles = search['principal_committees'][idx][n]['cycles']

            # assuming 1 principal committee per candidate per election cycle 
            # otherwise will select last principal committee listed...
            if year in cycles:
                principal_committees.loc[idx, 'CANDIDATE_ID'] =\
                    search.loc[idx, 'candidate_id']
                principal_committees.loc[idx, 'COMMITTEE_ID'] =\
                    search.loc[idx, 'principal_committees'][n]['committee_id']
                principal_committees['COMMITTEE_TYPE'] = 'H'
    
    return principal_committees

In [19]:
principal_committees_all = pd.DataFrame()

for state in states_and_districts():
    principal_committees_all =\
        principal_committees_all.append(get_principal_committees(
            house_results_2016_clean[house_results_2016_clean['STATE_ABBREVIATION'] == state]))

In [20]:
principal_committees_all.shape

(1349, 3)

In [21]:
# save the dataframes as a CSV file
principal_committees_all.to_csv('principal_committees_house_2016.csv', index=False)

## Combining Election Results with Principal Committee IDs
### Combine House Election Results with the candidate's corresponding Principal Committee ID

In [38]:
# Merge house results and principal committees keeping only those rows i.e. candidate IDs
# that appear in both (inner join).
combined = house_results_2016_clean.merge(principal_committees_all, how='inner',
                                           on='CANDIDATE_ID')

In [39]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1349 entries, 0 to 1348
Data columns (total 13 columns):
STATE_ABBREVIATION     1349 non-null object
D                      1349 non-null object
INCUMBENT              1349 non-null int64
CANDIDATE_NAME         1349 non-null object
CANDIDATE_ID           1349 non-null object
PARTY                  1349 non-null object
PRIMARY_VOTES          1110 non-null float64
PRIMARY_%              1109 non-null float64
GENERAL_VOTES          839 non-null float64
GENERAL_%              837 non-null float64
GE_WINNER_INDICATOR    1349 non-null int64
COMMITTEE_ID           1349 non-null object
COMMITTEE_TYPE         1349 non-null object
dtypes: float64(4), int64(2), object(7)
memory usage: 147.5+ KB


In [40]:
# save the dataframes as a CSV file
combined.to_csv('house_results_pc_2016.csv', index=False)

## Obtain Financial Reports for each Principal Committee
### Search for all Form 3 filed by each Principal Committee

In [25]:
# used to throttle API calls to FEC (limit 1000 per hour)
@sleep_and_retry
@limits(calls=15, period=60)
def url_call_committee_fin_reports(committee_id, year):
    url = ("https://api.open.fec.gov/v1/"
            "committee/{c}/reports/?"
            "page=1"
            "&year=2015"  
            "&year=2016"  
            "&api_key={api}"
            "&per_page=50"
            "&is_amended=False".format(c=committee_id,
                                       api=myapikey)
            )

    # 'requests.models.Response' type returned
    r = requests.get(url)

    # verify URL
    if r.status_code != 200:
        raise Exception('API response: {}'.format(r.status_code)) 

    # convert 'Response' object to dict
    r_dict = r.json()

    # convert dict to a Dataframe object and return Dataframe
    return pd.DataFrame.from_dict(r_dict['results'])

In [26]:
financial_reports = pd.DataFrame()

for com in combined['COMMITTEE_ID']:

    financial_reports =\
        financial_reports.append(url_call_committee_fin_reports(com, year), 
                                sort=True, ignore_index=True)

In [41]:
# save the dataframes as a CSV file
financial_reports.to_csv('financial_reports_house_2016.csv', index=False)

# THE END
### See data_analysis.ipynb for Data Analysis using the data obtained above.